In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [55]:
def soup_to_dico(_soup):
    temps = _soup.find('div', {"class" : "time__details"}).find_all('div')
    temps_preparation = temps[1].get_text()
    temps_repos = temps[3].get_text()
    temps_cuisson = temps[5].get_text()

    image = _soup.find('img',{"class":"lazyload mrtn-print-only"}).get("data-src")

    nb_personne = _soup.find("div",{"class":'mrtn-recette_ingredients-counter'}).get("data-servingsnb")

    difficulte = _soup.find_all('div',{"class": 'recipe-primary__item'})
    result = []
    for div in difficulte:
        # Trouver la balise <span> et extraire son texte
        span = div.find('span')
        if span:
            result.append(span)
    temps_total = result[0].get_text()
    difficulte= result[1].get_text()
    cout = result[2].get_text()

    instructions = _soup.find_all("div",{"class":'recipe-step-list__container'})
    etapes = []
    for i in range(len(instructions)):
        for p in instructions[i].find_all("p"):
            etapes.append(p.get_text())
    
    liste_ingredients = []

    ingredients = _soup.find_all("span",{"class":"card-ingredient-title"})
    ingredients_id = _soup.find_all("div",{"class":"card-ingredient-checkbox"})

    for i in range(len(ingredients)):
        nom = ingredients[i].find("span",{"class":"ingredient-name"}).get("data-ingredientnameplural")
        quantite_unite = ingredients[i].find("span", {"class":"card-ingredient-quantity"})
        quantite = quantite_unite.get("data-ingredientquantity")
        unite = quantite_unite.find("span", {"class":"unit"}).get("data-unitsingular")
        id_ingredient= ingredients_id[i].find("input").get("id")
        

        liste_ingredients.append({
            "nom":nom,
            "quantite": quantite,
            "unite":unite,
            "id": id_ingredient[6:]
        })

    recette_ = {
        "temps_preparation" : temps_preparation,
        "tems_repos" : temps_repos,
        "temps_cuisson": temps_cuisson ,
        "image" : image ,
        "nb_personne": nb_personne,
        "temps_total": temps_total ,
        "difficulte":difficulte,
        "cout": cout,
        "etapes": etapes,
        "ingredients": liste_ingredients
    }

    return recette_
    

Récupère les titres et les liens vers les entrées et les plats des recettes Top de marmiton

In [56]:
urls = [
    'https://www.marmiton.org/recettes/top-internautes-plat-principal.aspx',
    'https://www.marmiton.org/recettes/top-internautes-entree.aspx',
    "https://www.marmiton.org/recettes/top-internautes-dessert.aspx"
    ]

titres = []
liens = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    titre = soup.find_all('h4', {"class" : "recipe-card__title"})
    lien=soup.find_all('a', {"class" : "recipe-card-link"})

    titres.extend(titre)
    liens.extend(lien)

recettes = [] #titres, liens
for titre, lien in zip(titres, liens):
    recettes.append({
        "titre": titre.get_text().strip(),
        "lien": lien["href"]
    })

for recette in recettes:
    lien = recette['lien']
    id = re.findall(r'(\d+)', lien)[0]
    response_recette = requests.get(lien)
    soup_recette = BeautifulSoup(response_recette.content, "html.parser")
    recette['id'] = id
    recette.update(soup_to_dico(soup_recette))






Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [58]:
recettes[1]["ingredients"]

[{'nom': 'beurre', 'quantite': '125', 'unite': 'g'},
 {'nom': 'farine', 'quantite': '100', 'unite': 'g'},
 {'nom': 'poivre', 'quantite': '0', 'unite': ''},
 {'nom': 'sel', 'quantite': '0', 'unite': ''},
 {'nom': 'fromage râpé', 'quantite': '70', 'unite': 'g'},
 {'nom': 'muscade', 'quantite': '3', 'unite': 'pincée'},
 {'nom': 'thym', 'quantite': '0', 'unite': ''},
 {'nom': 'feuilles de laurier', 'quantite': '2', 'unite': ''},
 {'nom': 'vin rouge', 'quantite': '20', 'unite': 'cl'},
 {'nom': 'purée de tomate', 'quantite': '800', 'unite': 'g'},
 {'nom': 'carottes', 'quantite': '1', 'unite': ''},
 {'nom': 'oignons jaunes', 'quantite': '3', 'unite': ''},
 {'nom': 'lasagnes', 'quantite': '1', 'unite': 'paquet'},
 {'nom': 'ail', 'quantite': '2', 'unite': 'gousse'},
 {'nom': 'céleri', 'quantite': '1', 'unite': 'branche'},
 {'nom': 'boeuf haché', 'quantite': '600', 'unite': 'g'},
 {'nom': 'eau', 'quantite': '15', 'unite': 'cl'},
 {'nom': 'basilic', 'quantite': '0', 'unite': ''},
 {'nom': 'Parmes

In [53]:
url = "https://www.marmiton.org/recettes/recette_blanquette-de-veau-facile_19219.aspx"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

liste_ingredients =[]

ingredients_id = soup.find_all("div",{"class":"card-ingredient-checkbox"})
for i in range(len(ingredients_id)):
    id_ingredient= ingredients_id[i].find("input").get("id")
    liste_ingredients.append({"id": id_ingredient[6:]})

liste_ingredients

[{'id': '35'},
 {'id': '90'},
 {'id': '89'},
 {'id': '869'},
 {'id': '150'},
 {'id': '72'},
 {'id': '8'},
 {'id': '964'},
 {'id': '09'},
 {'id': '98'},
 {'id': '50'},
 {'id': '393'},
 {'id': '221'}]